In [1]:
# import necessary packages
# standard
import sys, os, glob
import numpy as np
import pandas as pd
import networkx as nx
from scipy import sparse
from tqdm.notebook import tqdm

# plotting
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from IPython.display import Javascript

# torch
import torch
import torch.nn.functional as F
from torch.nn import Linear

# torch geometric
from torch_geometric.utils import from_scipy_sparse_matrix, from_networkx
from torch_geometric.nn import (GAT, GCNConv, GATConv)
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import (NormalizeFeatures, RandomNodeSplit)
from torch_geometric.data import Data
from torch_geometric import seed_everything
from DGXutils import GetLowestGPU, GetFileNames

# sklearn
from sklearn.manifold import TSNE

# path
sys.path.append('../')

# custom
import utils.preprocessing as pp


/mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/torch_geometric/typing.py:85: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/torch_geometric/typing.py:96: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/torch_geometric/typing.py:112: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage

In [2]:
# load in data
network_path = "../data/networks/GTEx_BulkPENs/"
graph_names = GetFileNames(network_path, ".txt")

In [3]:
graph_names

['frontalCortex_top0.003pct.txt',
 'frontalCortex_top0.118pct.txt',
 'frontalCortex_top0.376pct.txt']

In [4]:
# convert graphs to node/edge lists and then nx objects
graphs = []
for name in tqdm(graph_names):
    # convert to nx object and append
    graphs.append(pp.convert_to_nx(network_path + name))
print(f"Processed {len(graphs)} graphs!")

  0%|          | 0/3 [00:00<?, ?it/s]

Processed 3 graphs!


In [5]:
# get example graph
example = graphs[1]
print(f"Graph has {len(example.nodes)} nodes and {len(example.edges)} edges.")

Graph has 41471 nodes and 50365 edges.


In [7]:
# visualize example graph
plt.figure(figsize=(40,40))
nx.draw_spring(example,
               with_labels=False,
               node_size=200)
plt.savefig("../data/images/test_graph.png")
plt.show()

KeyboardInterrupt: 

<Figure size 4000x4000 with 0 Axes>

In [14]:
# convert networks to torch geometric data objects
data_list = []
for graph in tqdm(graphs):
    data_list.append(from_networkx(graph))
print(f"Processed {len(data_list)} graphs!")

  0%|          | 0/3 [00:00<?, ?it/s]

Processed 3 graphs!


In [15]:
# create GAT
gat = GAT(in_channels=1, 
          hidden_channels=8,
          num_layers=2,
          out_channels=1)
gat

GAT(1, 1, num_layers=2)